# Projeto de Bases de Dados - Parte 3

### Docente Responsável

Prof. Daniel Pedro de Jesus Faria

### Grupo 40
<dl>
    <dt>18 horas (33.3%)</dt>
    <dd>ist1103811 Tiago Deane</dd>
    <dt>18 horas (33.3%)</dt>
    <dd>is1104145 Artur Krystopchuk</dd>
    <dt>18 horas (33.3%)</dt>
    <dd>ist193718 Guilherme Barata</dd>
<dl>

In [ ]:
%load_ext sql
%sql postgresql://db:db@postgres/db

# Empresa de comércio online

## 0. Carregamento da Base de Dados

Carregue o esquema de Base de Dados apresentado no Anexo A.

In [ ]:
%%sql

DROP TABLE IF EXISTS customer CASCADE;
DROP TABLE IF EXISTS orders CASCADE;
DROP TABLE IF EXISTS pay CASCADE;
DROP TABLE IF EXISTS employee CASCADE;
DROP TABLE IF EXISTS process CASCADE;
DROP TABLE IF EXISTS department CASCADE;
DROP TABLE IF EXISTS workplace CASCADE;
DROP TABLE IF EXISTS works CASCADE;
DROP TABLE IF EXISTS office CASCADE;
DROP TABLE IF EXISTS warehouse CASCADE;
DROP TABLE IF EXISTS product CASCADE;
DROP TABLE IF EXISTS contains CASCADE;
DROP TABLE IF EXISTS supplier CASCADE;
DROP TABLE IF EXISTS delivery CASCADE;

CREATE TABLE customer(
cust_no INTEGER PRIMARY KEY,
name VARCHAR(80) NOT NULL,
email VARCHAR(254) UNIQUE NOT NULL,
phone VARCHAR(15),
address VARCHAR(255)
);

CREATE TABLE orders(
order_no INTEGER PRIMARY KEY,
cust_no INTEGER NOT NULL REFERENCES customer,
date DATE NOT NULL
--order_no must exist in contains
);

CREATE TABLE pay(
order_no INTEGER PRIMARY KEY REFERENCES orders,
cust_no INTEGER NOT NULL REFERENCES customer
);

CREATE TABLE employee(
ssn VARCHAR(20) PRIMARY KEY,
TIN VARCHAR(20) UNIQUE NOT NULL,
bdate DATE,
name VARCHAR NOT NULL
--age must be >=18
);

CREATE TABLE process(
ssn VARCHAR(20) REFERENCES employee,
order_no INTEGER REFERENCES orders,
PRIMARY KEY (ssn, order_no)
);

CREATE TABLE department(
name VARCHAR PRIMARY KEY
);

CREATE TABLE workplace(
address VARCHAR PRIMARY KEY,
lat NUMERIC(8, 6) NOT NULL,
long NUMERIC(9, 6) NOT NULL,
UNIQUE(lat, long)
--address must be in warehouse or office but not both
);

CREATE TABLE office(
address VARCHAR(255) PRIMARY KEY REFERENCES workplace
);

CREATE TABLE warehouse(
address VARCHAR(255) PRIMARY KEY REFERENCES workplace
);

CREATE TABLE works(
ssn VARCHAR(20) REFERENCES employee,
name VARCHAR(200) REFERENCES department,
address VARCHAR(255) REFERENCES workplace,
PRIMARY KEY (ssn, name, address)
);

CREATE TABLE product(
SKU VARCHAR(25) PRIMARY KEY,
name VARCHAR(200) NOT NULL,
description VARCHAR,
price NUMERIC(10, 2) NOT NULL,
ean NUMERIC(13) UNIQUE
);

CREATE TABLE contains(
order_no INTEGER REFERENCES orders,
SKU VARCHAR(25) REFERENCES product,
qty INTEGER,
PRIMARY KEY (order_no, SKU)
);

CREATE TABLE supplier(
TIN VARCHAR(20) PRIMARY KEY,
name VARCHAR(200),
address VARCHAR(255),
SKU VARCHAR(25) REFERENCES product,
date DATE
);

CREATE TABLE delivery(
address VARCHAR(255) REFERENCES warehouse,
TIN VARCHAR(20) REFERENCES supplier,
PRIMARY KEY (address, TIN)
);

Crie as instruções para o seu preenchimento de forma consistente, garantindo que todas as consultas SQL e OLAP, apresentadas mais adiante, produzam um resultado não vazio. 

In [ ]:
%%sql

BEGIN;

INSERT INTO Employee (ssn, TIN, bdate, name) VALUES (123456789012, 987654321, '1990-01-01', 'Adams Brown');
INSERT INTO Employee (ssn, TIN, bdate, name) VALUES (987654321012, 123456789, '1995-05-10', 'Cook Davis');
INSERT INTO Employee (ssn, TIN, bdate, name) VALUES (987456321012, 892752342, '2003-06-14', 'Elvis Franklin');
INSERT INTO Employee (ssn, TIN, bdate, name) VALUES (123789456012, 321456798, '2000-04-20', 'Grant Hillary');
INSERT INTO Employee (ssn, TIN, bdate, name) VALUES (764589732694, 793425984, '2000-06-21', 'John Smith');
INSERT INTO Employee (ssn, TIN, bdate, name) VALUES (983979423983, 123141233, '2003-12-25', 'Gran Torino');

INSERT INTO Department (name) VALUES ('Computing');
INSERT INTO Department (name) VALUES ('Marketing');
INSERT INTO Department (name) VALUES ('Sales');
INSERT INTO Department (name) VALUES ('Design');
INSERT INTO Department (name) VALUES ('Shipping');

INSERT INTO Workplace (address, lat, long) VALUES ('1234-567 Main City', 10.1234, -34.5678);
INSERT INTO Workplace (address, lat, long) VALUES ('2345-678 Church City', 23.5678, -1.1234);
INSERT INTO Workplace (address, lat, long) VALUES ('3456-789 Deez City', 10.1234, -33.5678);
INSERT INTO Workplace (address, lat, long) VALUES ('4567-890 Nuzt City', 98.7654, -71.1234);

INSERT INTO Office (address) VALUES ('1234-567 Main City');
INSERT INTO Office (address) VALUES ('2345-678 Church City');

INSERT INTO Warehouse (address) VALUES ('3456-789 Deez City');
INSERT INTO Warehouse (address) VALUES ('4567-890 Nuzt City');

INSERT INTO works (ssn, name, address) VALUES (123456789012, 'Computing', '1234-567 Main City');
INSERT INTO works (ssn, name, address) VALUES (987654321012, 'Marketing', '2345-678 Church City');
INSERT INTO works (ssn, name, address) VALUES (987456321012, 'Sales', '2345-678 Church City');
INSERT INTO works (ssn, name, address) VALUES (123789456012, 'Design', '3456-789 Deez City');
INSERT INTO works (ssn, name, address) VALUES (123789456012, 'Shipping', '4567-890 Nuzt City');

INSERT INTO Customer (cust_no, name, email, phone, address) VALUES (1, 'Adams Brown', 'adams@gmail.com', '123-456-7890', '5678-901 Dragin City');
INSERT INTO Customer (cust_no, name, email, phone, address) VALUES (2, 'Cook Davis', 'cook@food.com', '987-654-3210', '1234-567 Main City');
INSERT INTO Customer (cust_no, name, email, phone, address) VALUES (3, 'John Smith', 'john@smith.com', '123-654-3210', '1234-567 Main City');
INSERT INTO Customer (cust_no, name, email, phone, address) VALUES (4, 'Gran Torino', 'gran@mha.com', '987-123-3210', '3456-789 Deez City');
INSERT INTO Customer (cust_no, name, email, phone, address) VALUES (5, 'Violet Evergardem', 've@gmail.com', '988-123-3210', '6543-789 Lookout Landing');

INSERT INTO orders (order_no, date, cust_no) VALUES (1, '2022-01-15', 1);
INSERT INTO orders (order_no, date, cust_no) VALUES (2, '2022-05-23', 2);
INSERT INTO orders (order_no, date, cust_no) VALUES (3, '2022-05-24', 3);
INSERT INTO orders (order_no, date, cust_no) VALUES (4, '2022-05-25', 4);
INSERT INTO orders (order_no, date, cust_no) VALUES (5, '2023-05-26', 1);
INSERT INTO orders (order_no, date, cust_no) VALUES (6, '2023-05-27', 2);
INSERT INTO orders (order_no, date, cust_no) VALUES (7, '2023-05-28', 3);
INSERT INTO orders (order_no, date, cust_no) VALUES (8, '2023-05-29', 4);

INSERT INTO process (ssn, order_no) VALUES (123789456012, 1);
INSERT INTO process (ssn, order_no) VALUES (123789456012, 2);
INSERT INTO process (ssn, order_no) VALUES (123789456012, 3);
INSERT INTO process (ssn, order_no) VALUES (123789456012, 4);
INSERT INTO process (ssn, order_no) VALUES (987654321012, 5);
INSERT INTO process (ssn, order_no) VALUES (987456321012, 6);
INSERT INTO process (ssn, order_no) VALUES (123456789012, 7);

INSERT INTO pay (order_no, cust_no) VALUES (1, 1);
INSERT INTO pay (order_no, cust_no) VALUES (2, 2);
INSERT INTO pay (order_no, cust_no) VALUES (3, 3);
INSERT INTO pay (order_no, cust_no) VALUES (7, 3);

INSERT INTO Product (sku, name, description, price) VALUES ('SKU001', 'The Dragon Slayer', 'A dragon slaying toy sword for your kids!', 80.00);
INSERT INTO Product (sku, name, description, price) VALUES ('SKU002', 'Hylian Shield', 'A replica of the shield from Legend of Zellda', 15.00);
INSERT INTO Product (sku, name, description, price) VALUES ('SKU003', 'Pure Nail', 'The newest nail clipper from NailClippersEnterprise', 3.50);
INSERT INTO Product (sku, name, description, price) VALUES ('SKU004', 'LEGU Millenium Falcon', 'The latest edition of LEGU Millenium Falcon from Star Wars', 500.00);
INSERT INTO Product (sku, name, description, price) VALUES ('SKU005', 'Philosophers Stone', 'The ideal item for alchemy!', 55.00);

INSERT INTO Supplier (TIN, name, address, sku, date) VALUES (123456789, 'NotBerserk', '6789-012 Slim City', 'SKU001', '2022-01-01');
INSERT INTO Supplier (TIN, name, address, sku, date) VALUES (123654789, 'Notindendo', '3456-789 Deez City', 'SKU002', '2022-02-01');
INSERT INTO Supplier (TIN, name, address, sku, date) VALUES (789235345, 'CherryTeam', '3456-789 Deez City', 'SKU003', '2022-03-01');
INSERT INTO Supplier (TIN, name, address, sku, date) VALUES (987654321, 'LEGU', '7890-123 Lego City', 'SKU004', '2022-04-01');
INSERT INTO Supplier (TIN, name, address, sku, date) VALUES (987654123, 'Alchemistz', '8901-234 FMA-HP City', 'SKU005', '2022-05-01');

INSERT INTO contains (order_no, sku, qty) VALUES (1, 'SKU001', 3);
INSERT INTO contains (order_no, sku, qty) VALUES (2, 'SKU002', 3);
INSERT INTO contains (order_no, sku, qty) VALUES (3, 'SKU003', 5);
INSERT INTO contains (order_no, sku, qty) VALUES (4, 'SKU004', 1);
INSERT INTO contains (order_no, sku, qty) VALUES (5, 'SKU005', 2);
INSERT INTO contains (order_no, sku, qty) VALUES (6, 'SKU001', 3);
INSERT INTO contains (order_no, sku, qty) VALUES (7, 'SKU002', 1);
INSERT INTO contains (order_no, sku, qty) VALUES (8, 'SKU003', 1);

INSERT INTO delivery (address, TIN) VALUES ('3456-789 Deez City', 123456789);
INSERT INTO delivery (address, TIN) VALUES ('4567-890 Nuzt City', 123654789);

COMMIT;


## 1. Restrições de Integridade

Apresente o código para implementar as seguintes restrições de integridade, se necessário, com recurso a extensões procedimentais SQL (Stored Procedures e Triggers):

(RI-1) Nenhum empregado pode ter menos de 18 anos de idade

In [ ]:
%%sql

ALTER TABLE employee
ADD CHECK (bdate + INTERVAL '18 years' <= CURRENT_DATE);

(RI-2) Um 'Workplace' é obrigatoriamente um 'Office' ou 'Warehouse' mas não pode ser ambos

In [ ]:
%%sql

-- Guarantees a workplace is either an Office or a Warehouse but never both
CREATE OR REPLACE FUNCTION valid_workplace() RETURNS TRIGGER AS $$
    BEGIN
        IF NEW.address IN (SELECT address FROM office) AND NEW.address IN (SELECT address FROM warehouse) THEN
            RAISE EXCEPTION 'Workplace cannot be Office and Warehouse';
        END IF;
        
        IF NEW.address NOT IN (SELECT address FROM office) AND NEW.address NOT IN (SELECT address FROM warehouse) THEN
            RAISE EXCEPTION 'Workplace needs to be Office or Warehouse';
        END IF;
        
        RETURN NEW;
    END;
$$ LANGUAGE plpgsql;

DROP TRIGGER IF EXISTS valid_workplace_trigger ON workplace;
CREATE CONSTRAINT TRIGGER valid_workplace_trigger AFTER INSERT OR UPDATE ON workplace DEFERRABLE INITIALLY DEFERRED
    FOR EACH ROW EXECUTE FUNCTION valid_workplace();

-- Guarantees an Office isn't a Warehouse
CREATE OR REPLACE FUNCTION valid_office() RETURNS TRIGGER AS $$
    BEGIN
        IF NEW.address IN (SELECT address FROM warehouse) THEN
            RAISE EXCEPTION 'Address already belongs to a warehouse';
        END IF;
        
        RETURN NEW;
    END;
$$ LANGUAGE plpgsql;

DROP TRIGGER IF EXISTS valid_office_trigger ON office;
CREATE CONSTRAINT TRIGGER valid_office_trigger AFTER INSERT OR UPDATE ON office DEFERRABLE INITIALLY DEFERRED
    FOR EACH ROW EXECUTE FUNCTION valid_office();

-- Guarantees a Warehouse isn't an Office
CREATE OR REPLACE FUNCTION valid_warehouse() RETURNS TRIGGER AS $$
    BEGIN
        IF NEW.address IN (SELECT address FROM office) THEN
            RAISE EXCEPTION 'Address already belongs to an office';
        END IF;
        
        RETURN NEW;
    END;
$$ LANGUAGE plpgsql;

DROP TRIGGER IF EXISTS valid_warehouse_trigger ON warehouse;
CREATE CONSTRAINT TRIGGER valid_warehouse_trigger AFTER INSERT OR UPDATE ON warehouse DEFERRABLE INITIALLY DEFERRED
    FOR EACH ROW EXECUTE FUNCTION valid_warehouse();

-- Guarantees that if an Office or Warehouse is deleted, the corresponding Workplace is deleted aswell
CREATE OR REPLACE FUNCTION delete_workplace() RETURNS TRIGGER AS $$
    BEGIN
        DELETE FROM Workplace w WHERE w.address = OLD.address;
        
        RETURN OLD;
    END;
$$ LANGUAGE plpgsql;

CREATE OR REPLACE TRIGGER warehouse_delete_workplace_trigger AFTER DELETE ON warehouse
    FOR EACH ROW EXECUTE FUNCTION delete_workplace();

CREATE OR REPLACE TRIGGER office_delete_workplace_trigger AFTER DELETE ON office
    FOR EACH ROW EXECUTE FUNCTION delete_workplace();

(RI-3) Uma 'Order' tem de figurar obrigatoriamente em 'Contains'.

In [ ]:
%%sql

-- Guarantees that an Order appears in Contains
CREATE OR REPLACE FUNCTION order_in_contains() RETURNS TRIGGER AS $$
    BEGIN
        IF NEW.order_no NOT IN (SELECT order_no FROM contains) THEN
            RAISE EXCEPTION 'Order % is not in contains', NEW.order_no;
        END IF;
        RETURN NEW;
    END;
$$ LANGUAGE plpgsql;

DROP TRIGGER IF EXISTS order_in_contains_trigger ON orders;
CREATE CONSTRAINT TRIGGER order_in_contains_trigger AFTER INSERT OR UPDATE ON orders DEFERRABLE INITIALLY DEFERRED
    FOR EACH ROW EXECUTE FUNCTION order_in_contains();

## 2. Consultas SQL

Apresente a consulta SQL mais sucinta para cada uma das seguintes questões

1) Qual o número e nome do(s) cliente(s) com maior valor total de encomendas pagas?  

In [ ]:
%%sql

SELECT cust_no, name
FROM customer JOIN pay USING (cust_no)
GROUP BY cust_no, name HAVING COUNT(*) >= ALL(
    SELECT COUNT(*)
    FROM customer JOIN pay USING (cust_no)
    GROUP BY cust_no
);

2. Qual o nome dos empregados que processaram encomendas em todos os dias de 2022 em que houve encomendas?

In [ ]:
%%sql

SELECT e.name
FROM employee e
WHERE e.ssn in (
    SELECT p.ssn
    FROM process p
    JOIN orders o ON p.order_no = o.order_no
    WHERE o.date >= '2022-01-01' AND o.date <= '2022-12-31'
    GROUP BY p.ssn
    HAVING COUNT(DISTINCT o.date) = (
    -- total de dias em 2022 que houve encomenedas
        SELECT COUNT(DISTINCT o.date)
        FROM orders o
        WHERE o.date >= '2022-01-01' AND o.date <= '2022-12-31'
    )
);

3. Quantas encomendas foram realizadas mas não pagas em cada mês de 2022?

In [ ]:
%%sql

SELECT EXTRACT(MONTH FROM dates.series) AS month, COALESCE(unpaid_orders, 0) AS unpaid_orders
FROM (
  SELECT generate_series('2022-01-01'::date, '2022-12-31'::date, '1 month') AS series
) AS dates
LEFT JOIN (
  SELECT EXTRACT(YEAR FROM o.date) AS year, EXTRACT(MONTH FROM o.date) AS month, COUNT(o.order_no) AS unpaid_orders
  FROM orders o
  LEFT JOIN pay p ON o.order_no = p.order_no
  WHERE p.order_no IS NULL
  GROUP BY year, month
) AS unpaid ON EXTRACT(YEAR FROM dates.series) = unpaid.year AND EXTRACT(MONTH FROM dates.series) = unpaid.month
ORDER BY dates.series;

## 3. Vistas

Crie uma vista que resuma as informações mais importantes sobre as vendas de produtos, combinando informações de diferentes tabelas do esquema de base de dados. A vista deve ter o seguinte esquema:

product_sales(sku, order_no, qty, total_price, year, month, day_of_month, day_of_week, city)

In [ ]:
%%sql

CREATE OR REPLACE VIEW product_sales AS
SELECT c.sku, c.order_no, c.qty, c.qty * p.price AS total_price,
       EXTRACT(YEAR FROM o.date) AS year,
       EXTRACT(MONTH FROM o.date) AS month,
       EXTRACT(DAY FROM o.date) AS day_of_month,
       EXTRACT(DOW FROM o.date) AS day_of_week,
       SUBSTRING(cm.address FROM '[0-9]{4}-[0-9]{3} (.+)$') AS city
FROM contains c
JOIN orders o ON c.order_no = o.order_no
JOIN customer cm ON o.cust_no = cm.cust_no
JOIN product p ON c.sku = p.sku
JOIN pay pa ON pa.order_no = o.order_no;

## 4. Desenvolvimento de Aplicação

### Explicação da arquitetura da aplicação web, incluindo um link para uma versão de trabalho e as relações entre os vários ficheiros na pasta web/arquivos

A aplicação web que desenvolvemos tem um índice principal (index.html) com 3 divisões: uma para produtos (Manage products), com as opções "Register product" e "List products", uma para fornecedores (Manage suppliers), com as opções "Register supplier" e "List suppliers" e uma para clientes (Manage customers), com as opções "Register customer" e "List customers".

Começando pela "Manage products", a opção "Register product" abre uma página (register_prod.html) com um formulário para preencher, pedindo o SKU, nome, descrição, preço e código EAN do novo produto, com um botão "save" para confirmar a criação do produto. A opção "List products" abre uma página  (list_products.html) que lista de todos os produtos, mostrando seu SKU, nome, preço e duas opções: "Edit" e "Order". A opção "Edit" abre uma página (update.html) que permite alterar a descrição e preço do produto e também apagar ele (todas as entradas em outras tabelas que dependem deste produto também são apagadas), enquanto a opção Order abre uma página (order_make.html) que pede o SKU, customer number, data e quantidade e cria a encomenda com estas informações (o seu order number será o maior order number mais um). A tabela "contains" também é atualizada de acordo com a nova encomenda criada.

Na divisão "Manage suppliers", a opção "Register supplier" abre uma página (register_sup.html) com um formulário que pede o TIN, endereço, nome, SKU e data e cria um novo fornecedor com essas informações. A opção "List suppliers" abre uma página (list_suppliers.html) que lista todos os fornecedores, mostrando todas as suas informações e ainda um botão "Delete", que permite remover aquele fornecedor (todas as entradas em outras tabelas dependentes daquele fornecedores também são apagadas).

Por fim, na divisão "Manage customers", a opção "Register customer" (register_cus.html) pede o nome, email, número de telemóvel e endereço e cria o cliente com estas informações (o customer number será o maior customer_number mais um). A opção "List customers" abre uma página (list_customers.html) que lista todos os clientes, mostrando todas as suas informações e ainda duas opções: "Delete" e "List orders". A opção "Delete" remove aquele cliente, enquanto a opção "List orders" abre uma página (list_orders.html) que lista todas as encomendas não pagas deste cliente, mostrando seu order number e a data que foi realizada, juntamente com uma opção "Pay" para realizar o pagamento da encomenda.

## 5. Consultas OLAP

Usando a vista desenvolvida para a Questão 3, escreva duas consultas SQL que permitam analisar:

1. As quantidade e valores totais de venda de cada produto em 2022, globalmente, por cidade, por mês, dia do mês e dia da semana

In [ ]:
%%sql

SELECT DISTINCT sku, city, dates.month, dates.day_of_month, dates.day_of_week, SUM(qty) AS quantity, SUM(total_price) AS total_price
FROM (
    SELECT
        EXTRACT(YEAR FROM d) AS year,
        EXTRACT(MONTH FROM d) AS month,
        EXTRACT(DAY FROM d) AS day_of_month,
        EXTRACT(DOW FROM d) AS day_of_week
    FROM
        generate_series('2022-01-01'::TIMESTAMP, '2022-12-31'::TIMESTAMP, '1 day') AS d
    ) AS dates
LEFT JOIN product_sales ps ON dates.year = ps.year
AND dates.month = ps.month
AND dates.day_of_month = ps.day_of_month
AND dates.day_of_week = ps.day_of_week
GROUP BY GROUPING SETS (
    (dates.month, dates.day_of_month, dates.day_of_week, sku, city),
    (dates.month, dates.day_of_month, dates.day_of_week, sku),
    (dates.month, dates.day_of_month, dates.day_of_week),
    (dates.month, dates.day_of_month),
    (dates.month),
    ())
ORDER BY dates.month, dates.day_of_month, dates.day_of_week, ps.sku, ps.city

2. O valor médio diário das vendas de todos os produtos em 2022, globalmente, por mês e dia da semana

In [ ]:
%%sql

SELECT dates.year, dates.month, dates.day_of_week, AVG(COALESCE(ps.total_price, 0)) AS average_daily_sales
FROM (
    SELECT
        EXTRACT(YEAR FROM d) AS year,
        EXTRACT(MONTH FROM d) AS month,
        EXTRACT(DOW FROM d) AS day_of_week
    FROM
        generate_series('2022-01-01'::TIMESTAMP, '2022-12-31'::TIMESTAMP, '1 day') AS d
    ) AS dates
LEFT JOIN product_sales ps ON dates.year = ps.year
AND dates.month = ps.month
AND dates.day_of_week = ps.day_of_week
GROUP BY GROUPING SETS (
  (dates.year, dates.month, dates.day_of_week),
  (dates.year, dates.month),
  ())
ORDER BY dates.year, dates.month, dates.day_of_week;

## 6. Índices

Indique, com a devida justificação, que tipo de índice(s), sobre qual(is) atributo(s) e sobre qual(is) tabela(s) faria sentido criar, de forma a agilizar a execução de cada uma das seguintes consultas: 

### 6.1
SELECT order_no
FROM orders 
JOIN contains USING (order_no) 
JOIN product USING (SKU) 
WHERE price > 50 AND 
EXTRACT(YEAR FROM date) = 2023

### Tipo de Índice, Atributos & Justificação

Para agilizar a execução desta consulta, devemos criar dois índices: um na coluna 'price' de 'product' e outro na coluna 'date' de 'orders'.
No caso do price temos um filtro de range (price > 50), algo que beneficia fortemente de um índice binary tree.  No caso de date também devemos criar um índice binary tree, pois date tem uma granularidade de dias e não anos, o que significa que, na realidade, nós queremos todas as linhas da tabela onde a data está entre 2023/01/01 e 2023/12/31, algo que é também um filtro de range.

Como order_no é primary key de orders (logo tem um índice por default), SKU é primary key de product (logo também tem um índice por default) e (order_no, SKU) é composite primary key de contains (logo há um composite index nestas colunas por default), as operações JOIN não iriam beneficiar de nenhum índice adicional.

### 6.2
SELECT order_no, SUM(qty*price)
FROM contains 
JOIN product USING (SKU) 
WHERE name LIKE ‘A%’ 
GROUP BY order_no;

### Tipo de Índice, Atributos & Justificação

Para agilizar esta consulta, devemos criar um índice b-tree na coluna 'name' de 'product', algo que irá melhorar o desempenho da cláusula WHERE, que usa o predicado "name LIKE 'A%'". Usar hash table só faria sentido se estivessemos à procura de uma palavra em especifico, logo usar b-tree, neste caso, é mais eficiente, uma vez que a binary tree fica ordenada alfabeticamente e só queremos saber as palavras que começam por A.

Como sku é primary key de product e (order_no, sku) é composite primary key de contains, estes já têm índices por default logo as operações JOIN e GROUP BY não iriam beneficiar de nenhum índice adicional.